In [40]:
import pandas as pd
from itertools import chain
import regex as re
from underthesea import word_tokenize


In [2]:
df = pd.read_csv("../data/procedure.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   procedure_name  2038 non-null   object
dtypes: object(1)
memory usage: 16.0+ KB


In [3]:
new_df = df.drop_duplicates()
new_df.to_csv('../data/new_procedure.csv', index=False)

In [4]:
df = pd.read_csv('../data/new_procedure.csv')
df.head()

,procedure_name
0,Thủ tục chuyển nhượng toàn bộ hoặc một phần dự...
1,Thủ tục cấp mới chứng chỉ hành nghề môi giới b...
2,Thủ tục cấp lại (cấp đổi) chứng chỉ hành nghề ...
3,Thủ tục thông báo nhà ở hình thành trong tương...
4,"Thủ tục cho thuê, thuê mua nhà ở xã hội thuộc ..."


In [15]:
f = open('../data/ongia_VA.txt', 'r', encoding='utf-8')
f_new = open('../data/procedure.txt', 'w', encoding='utf-8')
procedures = f.readlines()
for procedure in procedures:
    procedure = procedure.replace('\n','')
    f_new.write(procedure+'\t'+procedure)
    f_new.write('\n')
f_new.close()


In [9]:
def lcs(X, Y):
    # find the length of the strings
    m = len(X)
    n = len(Y)

    # declaring the array for storing the dp values
    L = [[None] * (n + 1) for i in range(m + 1)]

    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])

    return L[m][n]


In [16]:
def load_dict_values(dict_file):
    fin = open(dict_file, 'r', encoding='utf-8')
    lines = fin.readlines()
    fin.close()
    values_dict = []
    for line in lines:
        # print('line : ', line)
        items = line.strip().split('\t')
        # print('items : ', items)
        if len(items) == 2:
            words = items[0].lower().split()
            values_dict.append((words, items[1]))
    return values_dict


In [ ]:
list_procedure = load_dict_values('../data/procedure.txt')
list_procedure[:5]


In [19]:
def find_best_matching_in_dict(query, values_dict):
    query_words = query.lower().split()
    max_ratio = 0
    str_found = ""
    for words, v in values_dict:
        r = 1.0 * lcs(words, query_words) / len(words)
        if r > max_ratio:
            max_ratio = r
            str_found = v
    return str_found


In [61]:
mingg = find_best_matching_in_dict('kết hôn lại', load_dict_values('../data/procedure.txt'))
mingg

'Thủ tục đăng ký kết hôn'

In [33]:
def search_in_database(user_token):
    df = pd.read_csv('../data/new_procedure.csv')
    procedures = df[df.procedure_name.str.contains(user_token, na=False)]
    # flatten 2d list
    procedure_list = list(chain.from_iterable(procedures.values.tolist()))
    procedure_list = sorted(procedure_list, key=len)
    return procedure_list


In [69]:
def remove_dup(result_list):
    return list(dict.fromkeys(result_list))


In [34]:
kh = search_in_database('kết hôn')
kh

['Thủ tục đăng ký kết hôn',
 'Thủ tục đăng ký lại kết hôn',
 'Thủ tục đăng ký kết hôn lưu động',
 'Thủ tục Đăng ký kết hôn có yếu tố nước ngoài',
 'Thủ tục Đăng ký lại kết hôn có yếu tố nước ngoài',
 'Thủ tục Ghi vào Sổ hộ tịch việc kết hôn của công dân Việt Nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài',
 'Thủ tục Ghi vào Sổ hộ tịch việc ly hôn, hủy việc kết hôn của công dân Việt Nam đã được giải quyết tại cơ quan có thẩm quyền của nước ngoài']

In [36]:
user_sent = "đăng ký kết hôn địt mẹ m"
tmp = [user_sent]
tmp

['đăng ký kết hôn địt mẹ m']

In [75]:
def ranking_result(user_sent:str):
    values_dict = load_dict_values('../data/procedure.txt')
    best_matching = find_best_matching_in_dict(user_sent, values_dict)
    user_tokens = word_tokenize(user_sent)
    tmp =[]
    for token in user_tokens:
        tmp += search_in_database(token)
    unique_list = []
    dup_list = []
    for element in tmp:
        if element not in unique_list:
            unique_list.append(element)
        else:
            dup_list.append(element)
    dup_list = remove_dup(dup_list)
    for element in dup_list:
        if element in unique_list:
            unique_list.remove(element)
    if len(dup_list) == 0:
        return unique_list[:5]
    elif len(dup_list) <5 :
        n = 5 - len(dup_list)
        if best_matching:
            result = [best_matching] + dup_list + unique_list[:10]
            result = remove_dup(result)
            return result[:5]
        else :
            result = dup_list + unique_list[:5-n]
            return result[:5]
    else:
        if best_matching in dup_list:
            result = [best_matching] + dup_list
            result = remove_dup(result)
            return result[:5]
        else:
            return dup_list[:5]

In [78]:
mingg = ranking_result('khai sinh lại đăng ký')
mingg

['Thủ tục đăng ký lại khai sinh',
 'Thủ tục Đăng ký lại khai sinh có yếu tố nước ngoài',
 'Thủ tục đăng ký khai sinh',
 'Thủ tục đăng ký lại khai tử',
 'Thủ tục đăng ký lại kết hôn']